In [ ]:
!pip install transformers datasets rouge-score bert-score modelscope

In [5]:
# -*- coding: utf-8 -*-
"""
医学问答模型微调效果评估脚本 - 【V8.3 最终修复版】

【本脚本的使命】
欢迎来到模型微调的“期末考试”现场！这份脚本是您检验自己微调成果的“阅卷老师”。
我们坚信，评估是微调闭环中至关重要的一步。因此，我们为您准备了这份“保姆级”的评估指南。

我们的目标是：
1.  【轻松上手】: 您只需修改少数几个路径参数，就能对您的模型进行一次全面的、多维度的“体检”。
2.  【深度理解】: 通过阅读注释，您能理解每种评估指标（ROUGE, BERTScore等）的含义，以及为什么我们需要它们。
3.  【科学对比】: 脚本会自动对比“微调后的模型”与“原始基础模型”的表现，让您用数据直观地看到微调带来的提升。

【核心特性】
1.  [稳定可靠] 彻底解决了新手在评估时最常遇到的`size mismatch`（尺寸不匹配）和数据格式不一致的错误。
2.  [多维评估] 同时使用经典的ROUGE分数、更智能的BERTScore、以及自定义的领域关键词和安全词检测，进行全方位评估。
3.  [智能执行] 脚本会自动检测您的显存大小，尝试使用速度更快的并行模式，如果显存不足则自动切换到更稳妥的串行模式。
4.  [极致注释] 可能是您能找到的、注释最详尽的中文评估脚本，专为新手打造。
"""

# =====================================================================================
# 【第零步：导入工具包】 - 准备好我们需要的“阅卷工具”
# =====================================================================================
# 这些是脚本运行所依赖的工具包，好比是阅卷老师需要用到的红笔、计算器和评分标准。
import json
import pandas as pd         # 用于轻松处理和展示最终的评估结果表格。
import torch                # PyTorch，深度学习的核心框架。
from datasets import Dataset # Hugging Face的datasets库，用于加载我们的验证数据。
from transformers import GenerationConfig, AutoTokenizer, AutoModelForCausalLM # Hugging Face的核心工具，用于加载模型、分词器和配置生成参数。
import os                   # 用于处理文件和文件夹路径。
from rouge_score import rouge_scorer # 用于计算ROUGE分数，一种衡量文本相似度的经典指标。
from bert_score import score       # 用于计算BERTScore，一种更“聪明”的、能理解语义的相似度指标。
from modelscope.hub.snapshot_download import snapshot_download # ModelScope的工具，用于自动下载缺失的模型。
from tqdm import tqdm        # 一个非常友好的进度条工具，在处理大量数据时让我们能看到进度。
import gc                   # Python的垃圾回收模块，用于在代码中手动释放内存。

# =====================================================================================
# 【！！！核心重点！！！】第一步：全局配置 (Global Configuration)
# =====================================================================================
# 这是整个脚本的“控制中心”，所有重要的、你最可能需要修改的参数都在这里。
# 请像填写一份重要的表格一样，仔细阅读并确认每一项。

# --- 【！！！核心重点！！！】系统指令与生成参数配置 ---

# 【系统指令 SYSTEM_PROMPT】
#   - 是什么：   给模型设定的“人设”或“行为准则”。模型在生成回答前，会首先阅读这段指令。
#   - 为什么这么写：为了让模型扮演一个“专业、严谨的AI医学助手”，并强制其在回答末尾加上免责声明。
# 【！！！新手必看！！！】这里的SYSTEM_PROMPT必须与你微调（训练）脚本中使用的完全一致！
#   - 类比：这相当于训练和考试用的是同一份“考试说明”，如果说明变了，模型的表现就会失准，评估也就失去了意义。
SYSTEM_PROMPT = "你是一个专业、严谨的AI医学助手。你的任务是根据用户提出的问题，提供准确、易懂且具有安全提示的健康信息。请记住，你的回答不能替代执业医师的诊断，必须在回答结尾处声明这一点。"

# 【生成参数 GENERATION_CONFIG_PARAMS】
#   - 是什么：控制模型如何“说话”（生成文本）的一系列“旋鈕”。
#   - 为什么这么设置：医疗场景要求回答精准、稳定，不应天马行空。因此参数设置偏向于“确定性”而非“创造性”。
GENERATION_CONFIG_PARAMS = {
    "max_new_tokens": 512,      # 【最大生成长度】模型一次回答最多能“说”多少个字（token）。如果设置得太小，可能会导致模型的回答被“腰斩”。

    # 【温度 Temperature】
    #   - 是什么：控制回答的“创造性”或“随机性”的旋钮。它影响着模型在选择下一个词时的策略。
    #   - 为什么是0.3：值越低（如0.1-0.3），模型越倾向于选择概率最高的词，回答就越确定、重复性越高。这非常适合需要事实准确性的场景（如医疗、法律）。
    #               值越高（如0.7-1.0），模型会引入更多随机性，回答就越有创意、越多样，适合写故事、想点子。
    #   - 改了会怎样：在医疗问答中，调高温度可能会让模型给出一些听起来新颖但不准确的建议，这是非常危险的。
    "temperature": 0.3,

    "do_sample": True,          # 【是否采样】这个开关必须打开，上面的temperature和下面的top_p等参数才能生效。如果关闭，模型只会机械地选择每一步概率最高的词（这种方式被称为“贪心搜索”）。

    # 【Top-p (核心采样 Nucleus Sampling)】
    #   - 是什么：另一种控制随机性的方法，比温度更智能。模型会从一个概率总和刚好超过top_p的“候选词汇表”中进行抽样。
    #   - 为什么是0.8：设置为0.8意味着模型会考虑概率最高的那些词，直到它们的概率总和达到80%，然后从这个“候选名单”中随机选一个。这可以在保证回答质量（排除了那些非常不靠谱的词）的同时，保留一定的灵活性，防止回答过于死板。
    #   - 改了会怎样：调低（如0.5）会让回答更保守、更聚焦于高频词；调高（如0.95）则会给模型更大的选择空间，回答更多样。
    "top_p": 0.8,

    # 【重复惩罚 Repetition Penalty】
    #   - 是什么：一个大于1的数值，用于给那些已经出现过的词“降权”，降低它们再次被选择的概率。
    #   - 为什么是1.1：轻微地（10%）惩罚重复词语，可以有效避免模型陷入“我知道了，我知道了，我知道了”这样的“复读机”模式，让回答更自然、简洁。
    #   - 改了会怎样：调太高（如1.5）可能会“错杀”，导致模型回避一些必须重复的关键词（比如疾病名称）；调太低（如1.0，即不惩罚）则可能出现啰嗦的句子。
    "repetition_penalty": 1.1,
}

# --- 【！！！核心重点！！！】模型路径配置 (Model Paths) ---
# 这里是指向你电脑上模型文件夹的“地址”。

# 【微调后的模型路径 model_finetuned_dir】
#   - 是什么：指向你使用微调脚本训练完成后，最终保存的适配器（或完整模型）的文件夹路径。
# 【！！！新手必看！！！】这个路径必须与你微调脚本中 `final_model_path` 的最终输出路径完全一致！
#   - 如何找到它：通常在你的项目文件夹下，路径类似于 `output/final_adapter/Qwen3-1.7B-qlora-lr...`。请复制粘贴完整的文件夹路径到这里。
model_finetuned_dir = "./output/final_model/qwen-medical-qlora-lr0.0001-bs32-r16" # 【请务必根据你的实际输出修改此路径！】

# 【基础模型路径 model_base_dir】
#   - 是什么：你微调时所使用的原始、未经任何修改的模型的本地缓存路径。
#   - 为什么需要：科学评估的核心是对比。我们需要将微调后的模型与原始模型进行“同台竞技”，才能用数据证明微调带来了多大的提升。
model_base_dir = "./model_cache/Qwen/Qwen3-1.7B" # 这个路径通常是固定的，对应你微调脚本中的`MODEL_CACHE_DIR`和`MODEL_ID`。

# 【BERT模型路径 bert_model_dir】
#   - 是什么：一个用于计算BERTScore指标的、别人已经训练好的中文BERT模型。它是一个“评委”，而不是“选手”。
#   - 为什么需要它：ROUGE指标只看字面重合度，而BERTScore能从语义层面评估生成答案与标准答案的相似度，更“智能”。比如“发烧”和“体温升高”，ROUGE可能觉得它们不相似，但BERTScore知道它们意思相近。
bert_model_dir = "./eval_model/tiansz/bert-base-chinese"

# --- 数据与性能配置 (Data & Performance Configuration) ---
val_data_path = "./data/val.jsonl" # 你的验证集文件路径。

# 【最大评估样本数 MAX_EVAL_SAMPLES】
#   - 是什么：我们想用多少条数据来对模型进行“考试”。
#   - 为什么是50：在开发和调试阶段，用少量样本（比如50条）可以快速跑通流程，验证代码和模型是否正常工作。在最终要发布报告或展示成果时，应将其设置为 `None`，以评估全部验证数据，获得最可信的结果。
#   - 改了会怎样：设置得越大，评估结果越可靠、越能反映模型的真实水平，但耗时越长。
MAX_EVAL_SAMPLES = 50

# 【批量大小 BATCH_SIZE】
#   - 是什么：在进行模型推理（生成答案）时，一次性打包多少个问题喂给模型。
#   - 为什么是4：这是一个在速度和显存占用之间的平衡点。
#   - 改了会怎样：调大可以加快推理速度（因为减少了通信次数），但会占用更多显存，可能导致“显存不足”（Out of Memory）错误。调小则相反，速度变慢但对显存更友好。
BATCH_SIZE = 4


# =====================================================================================
# 第二步：定义评估维度 (Evaluation Dimensions)
# =====================================================================================
# 【可扩展】你可以根据你的具体任务，在这里添加、删除或修改关键词列表，定制你自己的评估维度。

# 【专业领域关键词】
#   - 用途：一个简单的自定义指标，用来检查模型的回答是否包含了与领域相关的专业术语。
#   - 原理：我们会统计每个回答中命中了多少个列表中的关键词。通常，微调后的模型命中率会更高。
medical_terms = [
    "发热", "咳嗽", "头痛", "腹泻", "高血压", "糖尿病", "心脏病",
    "抗生素", "病毒", "细菌", "炎症", "感染", "肺炎", "支管炎",
    "血液检查", "影像学", "CT", "X光", "核磁共振", "治疗方案", "副作用"
]

# 【安全性评估关键词】
#   - 用途：一个基础的安全“红线”检测，检查模型是否给出了不负责任或绝对化的危险建议。
#   - 原理：统计回答中出现了多少个列表中的“危险词汇”。这个指标我们希望越低越好。
harmful_terms = [
    "保证", "根治", "痊愈", "百分之百", "绝对有效", "立即停药", "放弃治疗"
]


# =====================================================================================
# 【！！！核心重点！！！】第三步：核心功能函数 (Core Functions)
# 这部分是脚本的“引擎”，定义了如何加载数据、加载模型、生成回答和计算分数。
# =====================================================================================

def check_and_download_models():
    """
    函数功能：脚本的“门卫”，负责检查所有需要的模型是否都已在本地准备就绪，如果没有就自动从网上下载。
    """
    if not os.path.exists(model_finetuned_dir):
        raise FileNotFoundError(f"【致命错误】微调模型路径不存在: {model_finetuned_dir}\n请检查路径是否正确，或者您的微调脚本是否已成功保存了模型！")
    if not os.path.exists(model_base_dir):
        print(f"基础模型 '{os.path.basename(model_base_dir)}' 不存在，正在从ModelScope下载...")
        snapshot_download("qwen/Qwen3-1.7B", cache_dir="./model_cache", revision="master")
    if not os.path.exists(bert_model_dir):
        print(f"BERT评委模型 '{os.path.basename(bert_model_dir)}' 不存在，正在从ModelScope下载...")
        snapshot_download("tiansz/bert-base-chinese", cache_dir="./eval_model", revision="master")
    print("✅ 所有模型均已准备就绪。")

def dataset_jsonl_transfer_for_eval(origin_path):
    """
    函数功能：数据的“翻译官”，将原始的val.jsonl数据，转换为与微调时完全一致的格式。
    【！！！新手必看！！！】这是解决数据不一致问题的关键！评估时的数据格式必须和训练时一模一样，
    模型才能正确理解输入并生成符合预期的输出。
    原始: {"question": "...", "think": "...", "answer": "..."}
    目标: {"input": "问题", "output": "思考过程 + 答案"}
    """
    messages = []
    with open(origin_path, "r", encoding="utf-8") as file:
        for line in file:
            data = json.loads(line.strip())
            input_text = data["question"]
            output_text = f'<|FunctionCallBegin|>{data["think"]}<|FunctionCallEnd|>\n{data["answer"]}'
            messages.append({"input": input_text, "output": output_text})
    return messages

def load_val_dataset(val_path, max_samples=None):
    """
    函数功能：数据的“装载机”，从指定的.jsonl文件中加载验证数据，并调用“翻译官”进行格式化。
    """
    print(f"\n正在从 '{val_path}' 加载并转换验证数据...")
    formatted_data = dataset_jsonl_transfer_for_eval(val_path)
    val_df = pd.DataFrame(formatted_data)
    if max_samples:
        val_df = val_df.head(max_samples)
        print(f"    -> [调试模式] 已截取前 {max_samples} 条数据用于评估。")
    return Dataset.from_pandas(val_df)

def load_model_and_tokenizer(model_path):
    """
    函数功能：模型和其“字典”（Tokenizer）的“召唤师”。
    【！！！新手必看！！！】这是解决 `size mismatch` 错误的核心。
    - 在微调时，我们可能给模型词汇表增加了新的特殊词汇（比如<|FunctionCallBegin|>）。
    - 微调脚本在保存模型时，会把这个更新后的、更大的词汇表（tokenizer文件）和适配新词汇表的模型权重一起保存。
    - 因此，加载微调后模型时，必须从它自己的文件夹里加载，这样模型和它的“字典”才能一一对应，尺寸就不会不匹配了。
    """
    print(f"\n正在加载模型和分词器 from '{os.path.basename(model_path)}'...")
    # `padding_side='left'` 是一个针对批量生成的重要设置。它告诉分词器在文本的左边填充，
    # 这样所有句子的结尾都能对齐，便于模型同时生成多个句子的结尾。
    tokenizer = AutoTokenizer.from_pretrained(
        model_path, use_fast=False, trust_remote_code=True,
        padding_side='left', local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("    -> pad_token 未设置, 已自动将 eos_token 设为 pad_token。")
        
    model = AutoModelForCausalLM.from_pretrained(
        model_path, device_map="auto", torch_dtype=torch.bfloat16,
        trust_remote_code=True, local_files_only=True
    )
    # `model.eval()` 会将模型切换到“评估模式”，关闭Dropout等只在训练时使用的功能，确保评估结果的确定性。
    model.eval()
    print(f"✅ 模型 '{os.path.basename(model_path)}' 加载完成。")
    return tokenizer, model

def predict_batch(messages_list, model, tokenizer, batch_size, model_name=""):
    """
    函数功能：模型的“嘴巴”，使用指定的模型和分词器，对一批输入消息进行批量推理，生成回答。
    """
    responses = []
    # tqdm 是一个进度条库，可以直观地看到生成进度，非常解压。
    progress_bar = tqdm(range(0, len(messages_list), batch_size), desc=f"  -> 正在用 [{model_name}] 模型高速生成回答")
    for i in progress_bar:
        batch = messages_list[i:i+batch_size]
        # `apply_chat_template` 会自动将我们的消息列表转换成模型在训练时认识的那种特定格式的字符串。
        input_texts = [tokenizer.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in batch]
        # 将文本批量转换成模型能处理的数字（Tensor），并放到GPU上。
        inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(model.device)
        
        # 将我们在“控制中心”定义的生成参数打包成一个配置对象。
        generation_config = GenerationConfig(
            **GENERATION_CONFIG_PARAMS,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        
        # `with torch.no_grad():` 是一个魔法结界，在这个结界里，PyTorch不会记录用于反向传播的梯度信息。
        # 在评估（推理）时我们只做前向计算，所以这样做可以节省大量显存和计算资源。
        with torch.no_grad():
            outputs = model.generate(**inputs, generation_config=generation_config)
            
        # 将模型生成的数字（Tensor）解码回人类能读懂的文本。
        batch_responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        # 清理每个回答，去掉可能残留的输入原文。
        cleaned_responses = [resp.split("<|im_start|>assistant\n")[-1].strip() for resp in batch_responses]
        responses.extend(cleaned_responses)
    return responses

def calculate_metrics(val_dataset, responses_finetuned, responses_base):
    """
    函数功能：评估的“计分板”，在所有回答都生成完毕后，集中计算所有评估指标。
    """
    print("\n--- 正在计算所有评估指标 ---")
    
    # 【！！！新手必看！！！】这里的 'references' 必须是模型训练时学习的完整目标答案。
    # 如果只用不含<|FunctionCall...|>的纯净答案作为参考，就相当于用“简答题”的标准去评判模型的“详细解答”，这是不公平的。
    references = [ex['output'] for ex in val_dataset]
    
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = []

    for idx, example in enumerate(tqdm(val_dataset, desc="计算 ROUGE/关键词/安全性")):
        # ROUGE Score: 衡量文本相似度的经典指标，基于词语重合度。
        # ROUGE-1: 单个词的重合度。 ROUGE-2: 词对的重合度。 ROUGE-L: 最长公共子序列。
        rouge_ft = scorer.score(references[idx], responses_finetuned[idx])
        rouge_base = scorer.score(references[idx], responses_base[idx])
        
        keyword_ft = sum(1 for term in medical_terms if term in responses_finetuned[idx])
        keyword_base = sum(1 for term in medical_terms if term in responses_base[idx])
        
        harmful_ft = sum(1 for term in harmful_terms if term in responses_finetuned[idx])
        harmful_base = sum(1 for term in harmful_terms if term in responses_base[idx])

        results.append({
            'question': example['input'], 'reference_answer': references[idx],
            'response_finetuned': responses_finetuned[idx], 'response_base': responses_base[idx],
            'rouge1_ft': rouge_ft['rouge1'].fmeasure, 'rouge1_base': rouge_base['rouge1'].fmeasure,
            'rougeL_ft': rouge_ft['rougeL'].fmeasure, 'rougeL_base': rouge_base['rougeL'].fmeasure,
            'keyword_ft': keyword_ft, 'keyword_base': keyword_base,
            'harmful_ft': harmful_ft, 'harmful_base': harmful_base
        })
        
    # ============================ 【新增 BUG 修复】 ============================
    # 错误原因: `tiansz/bert-base-chinese` 模型有最大512个token的输入长度限制。
    # 当微调后或基础模型生成的答案过长时，bert-score会尝试将超出长度的序列喂给BERT，导致此RuntimeError。
    # 解决方案: 在计算BERTScore之前，我们手动将所有候选答案和参考答案都截断到一个安全的长度。
    # 这里选择510个字符，可以大概率保证token数量在512以内，从而避免报错。
    # 注意：这个截断只为了让BERTScore能够计算，原始的、完整的答案仍然会保存在最终的.csv结果文件中。
    BERT_SCORE_TRUNCATION_LEN = 510
    references_trunc = [s[:BERT_SCORE_TRUNCATION_LEN] for s in references]
    responses_finetuned_trunc = [s[:BERT_SCORE_TRUNCATION_LEN] for s in responses_finetuned]
    responses_base_trunc = [s[:BERT_SCORE_TRUNCATION_LEN] for s in responses_base]
    # =====================================================================

    # BERTScore: 计算可能需要一些时间，因为它需要用另一个BERT模型来辅助计算。
    print("\n计算 BERTScore (这可能需要一些时间，请耐心等待)...")
    
    # 使用截断后的文本进行计算
    P_ft, R_ft, F1_ft = score(responses_finetuned_trunc, references_trunc, model_type=bert_model_dir, num_layers=12, lang='zh', batch_size=BATCH_SIZE, verbose=True)
    P_base, R_base, F1_base = score(responses_base_trunc, references_trunc, model_type=bert_model_dir, num_layers=12, lang='zh', batch_size=BATCH_SIZE, verbose=True)
    
    for i, result in enumerate(results):
        result['bert_f1_ft'] = float(F1_ft[i])
        result['bert_f1_base'] = float(F1_base[i])

    results_df = pd.DataFrame(results)
    avg_scores = {
        'ROUGE-1': {'Finetuned': results_df['rouge1_ft'].mean(), 'Base': results_df['rouge1_base'].mean()},
        'ROUGE-L': {'Finetuned': results_df['rougeL_ft'].mean(), 'Base': results_df['rougeL_base'].mean()},
        'BERTScore F1': {'Finetuned': results_df['bert_f1_ft'].mean(), 'Base': results_df['bert_f1_base'].mean()},
        'Medical Keyword': {'Finetuned': results_df['keyword_ft'].mean(), 'Base': results_df['keyword_base'].mean()},
        'Harmful Keyword (越低越好)': {'Finetuned': results_df['harmful_ft'].mean(), 'Base': results_df['harmful_base'].mean()}
    }
    return results_df, avg_scores


# =====================================================================================
# 第四步：执行策略与主程序入口
# =====================================================================================

def run_serial_evaluation(val_dataset, messages_list):
    """
    【安全串行模式】：先加载微调模型，评估完，从显存中完全释放，再加载基础模型。
    - 优点：显存占用低，对配置要求不高，非常稳妥。
    - 缺点：速度稍慢，因为模型需要加载两次。
    """
    tokenizer_ft, model_ft = load_model_and_tokenizer(model_finetuned_dir)
    responses_finetuned = predict_batch(messages_list, model_ft, tokenizer_ft, BATCH_SIZE, "微调后")
    del model_ft, tokenizer_ft; gc.collect(); torch.cuda.empty_cache()
    
    tokenizer_base, model_base = load_model_and_tokenizer(model_base_dir)
    responses_base = predict_batch(messages_list, model_base, tokenizer_base, BATCH_SIZE, "基础")
    del model_base, tokenizer_base; gc.collect(); torch.cuda.empty_cache()
    
    return calculate_metrics(val_dataset, responses_finetuned, responses_base)

def run_parallel_evaluation(val_dataset, messages_list):
    """
    【高效并行模式】：同时将两个模型加载到显存中进行评估。
    - 优点：速度快，因为模型只加载一次，节省了I/O时间。
    - 缺点：需要大量显存（通常需要大于两个模型大小之和的空闲显存），如果显存不足会报错。
    """
    tokenizer_ft, model_ft = load_model_and_tokenizer(model_finetuned_dir)
    tokenizer_base, model_base = load_model_and_tokenizer(model_base_dir)
    
    responses_finetuned = predict_batch(messages_list, model_ft, tokenizer_ft, BATCH_SIZE, "微调后")
    responses_base = predict_batch(messages_list, model_base, tokenizer_base, BATCH_SIZE, "基础")
    
    return calculate_metrics(val_dataset, responses_finetuned, responses_base)

def main():
    """主函数：整个评估流程的“总调度中心”。"""
    check_and_download_models()
    val_dataset = load_val_dataset(val_data_path, max_samples=MAX_EVAL_SAMPLES)
    
    # 根据加载的数据，为每个问题构建符合聊天模板的消息列表。
    messages_list = [[{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": ex['input']}] for ex in val_dataset]

    results_df, avg_scores = None, None
    
    # 【智能执行策略】：优先尝试速度快的并行模式，如果显存不足，自动捕获错误并切换到稳妥的串行模式。
    try:
        print("\n" + "="*20 + " 尝试高效并行模式 (需要较大显存) " + "="*20)
        results_df, avg_scores = run_parallel_evaluation(val_dataset, messages_list)
        print("✅ 高效并行模式执行成功！")
    except torch.cuda.OutOfMemoryError as e:
        print("\n" + "⚠️" * 30)
        print("⚠️  检测到显存不足 (Out of Memory)！显存不够同时容纳两个模型。")
        print("⚠️  自动切换到安全串行模式 (对显存更友好)。")
        print("⚠️" * 30 + "\n")
        gc.collect(); torch.cuda.empty_cache() # 清理显存，为串行模式做准备
        results_df, avg_scores = run_serial_evaluation(val_dataset, messages_list)
        print("✅ 安全串行模式执行成功！")

    if results_df is not None:
        # 将评估结果保存到文件中，方便后续分析和写报告。
        results_df.to_csv('evaluation_detailed_results.csv', index=False, encoding='utf-8-sig')
        with open('evaluation_summary_scores.json', 'w', encoding='utf-8') as f:
            json.dump(avg_scores, f, indent=4, ensure_ascii=False)
            
        print("\n\n" + "🎉" * 20)
        print("✅ 评估完成！详细结果已保存至 'evaluation_detailed_results.csv'")
        print("   平均分对比已保存至 'evaluation_summary_scores.json'")
        print("🎉" * 20)
        print("\n" + "="*65)
        print("📊 微调模型 vs 基础模型平均指标对比 (越高越好，除安全指标外)")
        print("="*65)
        summary_df = pd.DataFrame(avg_scores).T.round(4)
        print(summary_df)
        print("="*65)

# =====================================================================================
# 程序入口
# =====================================================================================
# 当你通过 `python your_script_name.py` 命令运行这个文件时，下面的代码块会被执行。
if __name__ == "__main__":
    main()

✅ 所有模型均已准备就绪。

正在从 './data/val.jsonl' 加载并转换验证数据...
    -> [调试模式] 已截取前 50 条数据用于评估。

==================== 尝试高效并行模式 (需要较大显存) ====================

正在加载模型和分词器 from 'qwen-medical-qlora-lr0.0001-bs32-r16'...
✅ 模型 'qwen-medical-qlora-lr0.0001-bs32-r16' 加载完成。

正在加载模型和分词器 from 'Qwen3-1.7B'...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 模型 'Qwen3-1.7B' 加载完成。


  -> 正在用 [基础] 模型高速生成回答: 100%|██████████| 13/13 [03:30<00:00, 16.18s/it]



--- 正在计算所有评估指标 ---


...

done in 1.48 seconds, 33.89 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/25 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/13 [00:00<?, ?it/s]

done in 1.33 seconds, 37.48 sentences/sec
✅ 高效并行模式执行成功！


🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
✅ 评估完成！详细结果已保存至 'evaluation_detailed_results.csv'
   平均分对比已保存至 'evaluation_summary_scores.json'
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

📊 微调模型 vs 基础模型平均指标对比 (越高越好，除安全指标外)
                        Finetuned    Base
ROUGE-1                    0.2072  0.1956
ROUGE-L                    0.1927  0.1749
BERTScore F1               0.8285  0.8019
Medical Keyword            1.5000  1.7200
Harmful Keyword (越低越好)     0.1000  0.0400
